In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pyproj
import folium
from folium.plugins import MarkerCluster

santa_rita_file_path = "C:/Users/Xiuchen Lu/Desktop/PRISM-vs.-my-RAINge-Log/UTM Coordinates of Rain Gauge Stations.xlsx"
santa_rita_shapefile_path = "C:/Users/Xiuchen Lu/Desktop/PRISM-vs.-my-RAINge-Log/SRER_BASEMAP/SRER_BASEMAP.shp"

santa_rita_location = pd.read_excel(santa_rita_file_path)

print("列名:", santa_rita_location.columns)

utm_zone = 12
proj_utm = pyproj.Proj(proj='utm', zone=utm_zone, datum='NAD83')
proj_wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

lon, lat = pyproj.transform(proj_utm, proj_wgs84, santa_rita_location['X-COORD'].values, santa_rita_location['Y-COORD'].values)
geometry = [Point(xy) for xy in zip(lon, lat)]

gdf = gpd.GeoDataFrame(santa_rita_location, geometry=geometry, crs=proj_utm.srs)
gdf = gdf.to_crs(epsg=4326)
print("GeoDataFrame:", gdf.head())

santa_rita_shape = gpd.read_file(santa_rita_shapefile_path)


gdf.to_file(r"C:\Users\Xiuchen Lu\Desktop\PRISM-vs.-my-RAINge-Log\rain_gauge_stations.geojson", driver="GeoJSON")


m = folium.Map(location=[lat.mean(), lon.mean()], zoom_start=12)
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain', attr='Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner', attr='Stamen Toner').add_to(m)
folium.TileLayer('Stamen Watercolor', attr='Stamen Watercolor').add_to(m)

folium.GeoJson(santa_rita_shape.to_crs(epsg=4326)).add_to(m)

marker_cluster = MarkerCluster().add_to(m)
for idx, row in gdf.iterrows():
    try:
        
        print(f"Adding marker at ({row.geometry.y}, {row.geometry.x}) with name {row.get('CURRENT STATION NAME', 'N/A')}")
        folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row.get('CURRENT STATION NAME', 'N/A')).add_to(marker_cluster)
    except KeyError as e:
        print(f"KeyError: {e}")

folium.LayerControl().add_to(m)
m.save("santa_rita_map.html")
m


列名: Index(['STATION CODE', 'CURRENT STATION NAME', 'X-COORD', 'Y-COORD'], dtype='object')
GeoDataFrame:   STATION CODE CURRENT STATION NAME   X-COORD    Y-COORD  \
0        DESST       Desert Station  509477.0  3527152.0   
1           NE            Northeast  515790.0  3530174.0   
2        LIMST            Limestone  517630.0  3526924.0   
3        FAGAN                Fagan  519250.5  3531291.5   
4        GRARI       Gravelly Ridge  504610.0  3522581.0   

                     geometry  
0  POINT (-115.48974 0.00029)  
1  POINT (-115.48974 0.00029)  
2  POINT (-115.48974 0.00029)  
3  POINT (-115.48974 0.00029)  
4  POINT (-115.48974 0.00029)  


C:\Users\Xiuchen Lu\AppData\Local\Temp\ipykernel_163368\94468421.py:24: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = pyproj.transform(proj_utm, proj_wgs84, santa_rita_location['X-COORD'].values, santa_rita_location['Y-COORD'].values)


Adding marker at (0.00028753835629061653, -115.4897374346761) with name Desert Station
Adding marker at (0.00028778363029937623, -115.48973683623298) with name Northeast
Adding marker at (0.00028751892738538567, -115.48973666241628) with name Limestone
Adding marker at (0.0002878740818847708, -115.48973650814078) with name Fagan
Adding marker at (0.0002871666729544361, -115.48973789590067) with name Gravelly Ridge
Adding marker at (0.0002872119655896576, -115.48973724375529) with name Muhlenbergia
Adding marker at (0.00028691410514505383, -115.48973729345796) with name Rodent
Adding marker at (0.00028677284546313843, -115.48973729855439) with name Road
Adding marker at (0.0002868070676261287, -115.48973742254178) with name Encl. No. 131
Adding marker at (0.0002868715126029409, -115.48973784797214) with name Amado
Adding marker at (0.0002866534171111938, -115.48973753498186) with name Enclosure 41
Adding marker at (0.0002865256529389256, -115.48973732175506) with name Whitehouse
Adding 